**실습에 필요한 libraries 들 설치 및 import**

In [1]:
!pip install mysql-connector-python

   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.4 MB 653.6 kB/s eta 0:00:24
   - -------------------------------------- 0.6/15.4 MB 6.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.7/15.4 MB 19.3 MB/s eta 0:00:01
   ----------- ---------------------------- 4.3/15.4 MB 25.2 MB/s eta 0:00:01
   ------------- -------------------------- 5.1/15.4 MB 25.2 MB/s eta 0:00:01
   ------------------ --------------------- 7.2/15.4 MB 28.8 MB/s eta 0:00:01
   ----------------------- ---------------- 8.9/15.4 MB 31.7 MB/s eta 0:00:01
   ----------------------------- ---------- 11.3/15.4 MB 50.4 MB/s eta 0:00:01
   ------------------------------- -------- 12.3/15.4 MB 43.5 MB/s eta 0:00:01
   --------------------------------- ------ 12.8/15.4 MB 36.3 MB/s eta 0:00:01
   --------------------------------- ------ 13.0/15.4 MB 34.4 MB/s eta 0:00:0

In [2]:
import mysql.connector
import sys
import pandas as pd
import matplotlib.pyplot as plt

**database 연결 정보 입력**

In [47]:
# Connect to the database
host = "147.46.15.238" # database 를 생성/연결할 서버 IP
port = "7000"
user = "DS2024_0024"
password = "DS2024_0024"
database = "DS2024_0024"

# mysql 을 서버에서 활용할 수 있도록 연결해준다.
mysql_con = mysql.connector.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database
    )

# Cursor 를 통해 서버의 database 와 커뮤니케이트 한다.
cur = mysql_con.cursor(dictionary=True) # dictionary

**load SQL Dump file**

get_dump 같은 경우, 처음 database 에 필요한 tables 생성시 호출하고 특이 케이스로 삭제되지 않는 한, 다시 호출할 필요없음.

In [48]:
# mysql_connector 로 mysql 서버를 호스트에 연결한 후 제공된 .sql file 을 읽어와 호스트에 database 를 생성하고 필요한 table 을 생성한다.

def get_dump(mysql_con, filename):

    query = ""
    try:
        with mysql_con.cursor() as cursor: # cursor 선언
            for line in open(filename, 'r', encoding='utf8'): # .sql 파일 열기
                if line.strip():
                    line = line.strip()
                    
                    # query 를 끝마치는 ";" 가 등장하기 전까지 파일을 읽으며 query 라는 변수에 저장 후 cursor 을 통해 서버의 database 로 송출
                    if line[-1] == ";":
                        query += line 
                        cursor.execute(query) # 서버의 database 로 보냄
                        query = ""
                    else:
                        query += line

    except Warning as warn:
        print(warn)
        sys.exit()

get_dump(mysql_con, 'ds2_database.sql')

In [49]:
# test connection
query = "SHOW tables;"     # 서버의 database 에 보내고 싶은 sql query
cur.execute(query)         # cur.execute() 을 통해 query 를 서버로 송출
a = cur.fetchall()         # cur.fetchall() 을 통해 query 실행 후 결과값 반환
print(a)

[{'Tables_in_DS2024_0024': 'customer'}, {'Tables_in_DS2024_0024': 'geolocation'}, {'Tables_in_DS2024_0024': 'order_info'}, {'Tables_in_DS2024_0024': 'order_item'}, {'Tables_in_DS2024_0024': 'order_payment'}, {'Tables_in_DS2024_0024': 'order_review'}, {'Tables_in_DS2024_0024': 'product'}, {'Tables_in_DS2024_0024': 'seller'}]


In [50]:
'''
    SQL query 를 받아 해당 query를 보내고 그 결과 값을 dataframe으로 저장해 return 해주는 함수
'''
def get_output(query):
    cur.execute(query)
    out = cur.fetchall()
    df = pd.DataFrame(out)
    return df

**Connection Test**: 테이블에 들어있는 정보를 모두 확인하고 각 테이블의 사이즈를 확인한다. ".head()" 를 통해서 앞쪽 n개의 row 만 출력할 수 있다.

In [9]:
# test connection
query = "SHOW COLUMNS FROM customer;"
get_output(query)

,Field,Type,Null,Key,Default,Extra
0,customer_id,varchar(255),NO,PRI,None,
1,customer_unique_id,text,YES,,None,
2,customer_zip_code_prefix,text,YES,,None,
3,customer_city,text,YES,,None,
4,customer_state,text,YES,,None,


In [10]:
query = "SELECT * FROM seller;"
get_output(query).head() # dataframe.head(n=5) : 앞쪽 n=5개 row 출력, SELECT * FROM seller LIMIT 5 와 동일

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,004c9cd9d87a3c30c522c48c4fc07416,14940,ibitinga,SP
1,00ee68308b45bc5e2660cd833c3f81cc,3333,sao paulo,SP
2,00fc707aaaad2d31347cf883cd2dfe10,87025,maringa,PR
3,01266d4c46afa519678d16a8b683d325,80250,curitiba,PR
4,0176f73cc1195f367f7b32db1e5b3aa8,14940,ibitinga,SP


In [11]:
query = "SELECT * FROM product;"
get_output(query).head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,009af1277432f1a05742df69cdf72d1e,brinquedos,58.0,916.0,3.0,700.0,20.0,35.0,20.0
1,00baba5b58e274d0332a0c8a0a66f877,perfumaria,27.0,406.0,4.0,200.0,18.0,7.0,12.0
2,00faa46f36261af8bbf3a4d37fa4841b,fashion_bolsas_e_acessorios,49.0,293.0,1.0,100.0,21.0,9.0,14.0
3,0152f69b6cf919bcdaf117aa8c43e5a2,cama_mesa_banho,40.0,116.0,1.0,250.0,40.0,4.0,30.0
4,0193fc18d9a6e9b9b0b3a41768fefa12,bebes,59.0,381.0,6.0,500.0,39.0,6.0,27.0


In [12]:
query = "SELECT * FROM geolocation;"
get_output(query).head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1026,-23.539657,-46.631884,são paulo,SP
1,1031,-23.541906,-46.635088,sao paulo,SP
2,11010,-23.933573,-46.329625,santos,SP
3,11030,-23.984858,-46.302070,santos,SP
4,11045,-23.969746,-46.321939,santos,SP


In [13]:
query = "SELECT * FROM order_info;"
get_output(query).head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,005d9a5423d47281ac463a968b3936fb,a2be07c22a9d2071f65b6770e47f540b,delivered,2017-10-18 12:14:29,2017-10-18 12:28:16,2017-10-24 14:56:52,2017-10-27 19:19:43,2017-11-17
1,0095790a64527ec83aeaaf99023c050e,6081c7acb40f63731b3c982857c79581,delivered,2017-11-15 13:36:30,2017-11-15 13:50:26,2017-11-16 19:44:48,2017-12-07 21:46:56,2017-12-11
2,00b30bb163474583c14db1689259cf4d,b626b511cecb256e0d1514d883084a38,delivered,2018-01-31 13:57:27,2018-01-31 14:13:31,2018-02-06 17:47:50,2018-02-17 18:36:59,2018-03-05
3,00c9d0257381c7e9ec13b4505f25b2cf,d14c86b99a98e85b5c256c081e3cea03,delivered,2018-04-18 08:57:18,2018-04-18 09:11:59,2018-04-20 22:58:35,2018-04-27 22:26:41,2018-05-18
4,00d9ac423c9ad6781c0ed04e87666691,64a495213f59f3050ccd2ed250d88dde,delivered,2017-10-03 11:54:16,2017-10-03 12:07:19,2017-10-03 20:28:19,2017-10-24 23:03:41,2017-11-03


In [14]:
query = "SELECT * FROM order_item;"
get_output(query).head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,005d9a5423d47281ac463a968b3936fb,1,fb7a100ec8c7b34f60cec22b1a9a10e0,d98eec89afa3380e14463da2aabaea72,2017-10-24 12:28:16,49.99,18.12
1,005d9a5423d47281ac463a968b3936fb,2,4c3ae5db49258df0784827bdacf3b396,d98eec89afa3380e14463da2aabaea72,2017-10-24 12:28:16,24.99,13.58
2,005d9a5423d47281ac463a968b3936fb,3,4c3ae5db49258df0784827bdacf3b396,d98eec89afa3380e14463da2aabaea72,2017-10-24 12:28:16,24.99,13.58
3,0095790a64527ec83aeaaf99023c050e,1,e8c6039a25765995ac7c1ec2cbef5765,3d871de0142ce09b7081e2b9d1733cb1,2017-11-21 13:50:26,19.90,42.76
4,00b30bb163474583c14db1689259cf4d,1,3552627a68384dc559f0fd4cce173269,3c487ae8f8d7542beff5788e2e0aea83,2018-02-06 14:13:31,189.90,26.61


In [15]:
query = "SELECT * FROM order_payment;"
get_output(query).head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,005d9a5423d47281ac463a968b3936fb,1,credit_card,10,145.26
1,0095790a64527ec83aeaaf99023c050e,1,credit_card,6,62.66
2,00b30bb163474583c14db1689259cf4d,1,credit_card,10,216.51
3,00c9d0257381c7e9ec13b4505f25b2cf,1,credit_card,7,387.57
4,00d9ac423c9ad6781c0ed04e87666691,1,credit_card,3,379.20


In [16]:
query = "SELECT * FROM order_review;"
get_output(query).head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_data,review_answer_timestamp
0,009962a4d3aef65d5dcf73b20fc47a74,152f562c8516765cdd5f868c22f3c96a,5,None,None,2018-06-26,2018-06-27 01:48:38
1,012a2c63edb084a590a56271f452e0dd,a5fd46480530fe84051edbf96b8beb9c,5,None,None,2018-04-11,2018-04-12 15:32:35
2,012f4d3e86e76a981b2891f33865eba3,5a3bccb60104b1251c6a39e051c428ba,4,8,"Gostei do produto, tive dificuldade para monta...",2018-08-23,2018-08-24 10:45:34
3,015ac3de931c36686ce859d0564305b5,70863e8ef99613bbc8f854807d187ea7,3,None,None,2018-02-28,2018-03-02 12:26:10
4,01e55995b828eb9eac03aa16f1a88663,8dae9d1421512248b0b7113ec0d7554a,4,MUITO BOM,"PRODUTO ÓTIMO, LACRADO E CHEGOU ANTES DO PRAZO.",2018-05-25,2018-05-25 23:44:01


In [17]:
query = "SELECT * FROM seller;"
get_output(query).head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,004c9cd9d87a3c30c522c48c4fc07416,14940,ibitinga,SP
1,00ee68308b45bc5e2660cd833c3f81cc,3333,sao paulo,SP
2,00fc707aaaad2d31347cf883cd2dfe10,87025,maringa,PR
3,01266d4c46afa519678d16a8b683d325,80250,curitiba,PR
4,0176f73cc1195f367f7b32db1e5b3aa8,14940,ibitinga,SP


In [18]:
# Table Size

query = "SELECT COUNT(customer_id) as num_customer FROM customer;"
print(get_output(query))

query = "SELECT COUNT(geolocation_zip_code_prefix) as num_geolocation FROM geolocation;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_order FROM order_info;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_item FROM order_item;"
print(get_output(query))

query = "SELECT COUNT(order_id) as num_pay FROM order_payment;"
print(get_output(query))

query = "SELECT COUNT(review_id) as num_review FROM order_review;"
print(get_output(query))

query = "SELECT COUNT(product_id) as num_product FROM product;"
print(get_output(query))

query = "SELECT COUNT(seller_id) as num_seller FROM seller;"
print(get_output(query))

   num_customer
0           852
   num_geolocation
0              607
   num_order
0        852
   num_item
0      1074
   num_pay
0      906
   num_review
0         921
   num_product
0          772
   num_seller
0         394


**SQL Query Format in Python**

In [ ]:
"\ 를 사용해서 query format 을 맞춰준다."

# 예시)

# query = "SELECT *\
#          FROM (SELECT col1, col2\
#               FROM table1 JOIN table2 using (col3)\
#               WHERE condition\
#               GROUP BY col3\
#               HAVING condition)\
#          WHERE condition;"

Tips

- LIMIT : table 을 위에서 부터 원하는 row 수 만큼 잘라서 가져온다 
- 예시) SELECT * FROM customer LIMIT 5; \
       -> customer table 에서 레코드를 가져오는데 상위 5개만 가져온다

- SUM(condition) : table 의 특정 attribute 이 condition 과 부합할 시 1 로 리턴되고 SUM() 을 통해서 condition 과 부합하는 row 의 수를 리턴한다 
- 예시) SUM('product_category_name' = 'home_appliances') \
       -> product table 에서 'product_category_name' attribute 기준, 'home_appliances' 인 레코드들의 총 개수를 리턴한다

**Question 0-1**: customers 중 "rio" 로 시작되는 도시에 거주 중인 사람들의 unique_id 를 출력한다.


In [23]:
query = "SELECT * FROM customer"
df = get_output(query)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               852 non-null    object
 1   customer_unique_id        852 non-null    object
 2   customer_zip_code_prefix  852 non-null    object
 3   customer_city             852 non-null    object
 4   customer_state            852 non-null    object
dtypes: object(5)
memory usage: 33.4+ KB


In [26]:
df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,0036875772a8eeb391b8cd7226283fdd,a73c5464692d3bc09d64e3edefae9015,9270,santo andre,SP
1,003ec17a1389fa302f3a0423dddd8650,1b6e96ed99cb8d135efe220d761bbd67,13190,monte mor,SP
2,0219be9d41699276f5d2010b4e758d51,51d91326ffc0688da14a001de38c264b,75712,catalao,GO
3,02447d437f30e908a881b92fdca09e41,615922c430abddd368735199c38e735d,27936,macae,RJ
4,0254fef29444edf8ad023adf1baac1bd,08e5b38d7948d37fbb2a59fc5e175ab1,58043,joao pessoa,PB


In [36]:
df[df.customer_city.str.startswith('rio')][['customer_unique_id']]

,customer_unique_id
6,c76762dfb642ac154475239639f7f8f4
11,a40096fc0a3862e9e12bc55b5f8e6ab2
26,2004ba7732339d9be65f2236d8887ae4
41,e016365b624d1755f3e2d0e8f1631ebb
66,c25d2fb6d22e04ce43c3652e3cacadad
...,...
771,64df076fb557552498049bb07ed1cd5f
795,a40096fc0a3862e9e12bc55b5f8e6ab2
804,935b9c5a3162185f88dac06d8d08d623
806,935b9c5a3162185f88dac06d8d08d623


**Question 0-2**: delivered 되지 않은 주문들의 order_id 및 order_status 를 출력한다.


In [29]:
query = "select * from order_info"
order_info = get_output(query)

In [31]:
order_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       852 non-null    object        
 1   customer_id                    852 non-null    object        
 2   order_status                   852 non-null    object        
 3   order_purchase_timestamp       852 non-null    datetime64[ns]
 4   order_approved_at              833 non-null    datetime64[ns]
 5   order_delivered_carrier_date   822 non-null    datetime64[ns]
 6   order_delivered_customer_date  814 non-null    datetime64[ns]
 7   order_estimated_delivery_date  852 non-null    datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 53.4+ KB


In [30]:
order_info.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,005d9a5423d47281ac463a968b3936fb,a2be07c22a9d2071f65b6770e47f540b,delivered,2017-10-18 12:14:29,2017-10-18 12:28:16,2017-10-24 14:56:52,2017-10-27 19:19:43,2017-11-17
1,0095790a64527ec83aeaaf99023c050e,6081c7acb40f63731b3c982857c79581,delivered,2017-11-15 13:36:30,2017-11-15 13:50:26,2017-11-16 19:44:48,2017-12-07 21:46:56,2017-12-11
2,00b30bb163474583c14db1689259cf4d,b626b511cecb256e0d1514d883084a38,delivered,2018-01-31 13:57:27,2018-01-31 14:13:31,2018-02-06 17:47:50,2018-02-17 18:36:59,2018-03-05
3,00c9d0257381c7e9ec13b4505f25b2cf,d14c86b99a98e85b5c256c081e3cea03,delivered,2018-04-18 08:57:18,2018-04-18 09:11:59,2018-04-20 22:58:35,2018-04-27 22:26:41,2018-05-18
4,00d9ac423c9ad6781c0ed04e87666691,64a495213f59f3050ccd2ed250d88dde,delivered,2017-10-03 11:54:16,2017-10-03 12:07:19,2017-10-03 20:28:19,2017-10-24 23:03:41,2017-11-03


In [33]:
order_info.order_status.unique()

array(['delivered', 'canceled', 'shipped', 'unavailable', 'processing',
       'invoiced'], dtype=object)

In [35]:
order_info[order_info.order_status != 'delivered'][['order_id','order_status']]

,order_id,order_status
5,00ff0cf5583758e6964723e42f111bf4,canceled
9,02e723e8edb4a123d414f56cc9c4665e,canceled
10,02ec4da9d03014f06d711d60eb37cc22,shipped
11,03310aa823a66056268a3bab36e827fb,canceled
41,0d3adebce4bebc1f80a7f36e9833f497,canceled
42,0e6dbbc7484dc1ee0392c9ca801035c2,canceled
77,1ad22e16129698e1a3cc11e4350d2cb7,canceled
159,37def1f304d4a5cfd597d66b3a778e9e,shipped
165,3a3aed58b078ba2d2105cd76ce7a76d7,canceled
219,47282cfe5747c1c19920f090e491d285,canceled


**Question 0-3**: 평점 3점 미만을 받은 주문들의 product_id 와 category 정보를 출력한다. 단, product_photos_qty 가 0 이 아닌 것들만 출력한다.

In [51]:
query = "select * from order_review"
order_review = get_output(query)
query = "select * from order_item"
order_item = get_output(query)
query = 'select * from product'
product = get_output(query)

In [56]:
df = pd.merge(pd.merge(order_item, order_review),product)

In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1153 entries, 0 to 1152
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   order_id                    1153 non-null   object        
 1   order_item_id               1153 non-null   int64         
 2   product_id                  1153 non-null   object        
 3   seller_id                   1153 non-null   object        
 4   shipping_limit_date         1153 non-null   datetime64[ns]
 5   price                       1153 non-null   float64       
 6   freight_value               1153 non-null   float64       
 7   review_id                   1153 non-null   object        
 8   review_score                1153 non-null   int64         
 9   review_comment_title        116 non-null    object        
 10  review_comment_message      434 non-null    object        
 11  review_creation_data        1153 non-null   datetime64[n

In [60]:
df[(df.product_photos_qty != 0) & (df.review_score<3)][['product_id','product_category_name']]

,product_id,product_category_name
0,fb7a100ec8c7b34f60cec22b1a9a10e0,brinquedos
1,4c3ae5db49258df0784827bdacf3b396,bebes
2,4c3ae5db49258df0784827bdacf3b396,bebes
7,bb1fe952bdc16ef8974d4ca11aec19b7,beleza_saude
13,a87567bdce60de6f901e3d7c6d8df820,cama_mesa_banho
...,...,...
1131,99f5b2240b82bf9387e5cf2fc0335b08,informatica_acessorios
1134,0ec7b6a863e219792bc4d50d3a62a839,cama_mesa_banho
1135,0ec7b6a863e219792bc4d50d3a62a839,cama_mesa_banho
1146,40a1e3c65a0bcf6f4ebba840a8156ba2,None


**Question 1-1**: 가장 많은 주문량 (주문 건수, order_id 의 개수) 을 가지는 상품 카테고리 5종의 이름 (product_category_name) 및 각 카테고리에 대한 product_id 의 수 (num_prod_ids 로 지정, 중복 허용) 를 주문량 기준 내림차순으로 출력한다. Category 정보가 없는 상품은 포함하지 않는다. (내림차순으로 정렬시 num_prod_ids 가 같으면 상품 카테고리 이름의 오름차순으로 정렬한다.)

In [ ]:
query = ""
print(get_output(query))

**Question 1-2**: 주문량이 가장 적은 상품 카테고리 5종의 이름 및 각 카테고리에 속하는 product_id 의 수 (num_prods_ids 로 지정) 를 주문량에 대한 오름차순으로 출력한다. (오름차순으로 정렬시 num_prod_ids 가 같을 경우 상품 카테고리 이름의 오름차순으로 정렬한다)

In [ ]:
query = ""
print(get_output(query))

**Question 2**: order_info 테이블의 "order_purchase_timestamp" column 에는 상품 주문 날짜에 대한 정보가 (YYYY-MM-DD hh:mm:ss) 형태로 담겨있다. 상품 주문량을 월별로 합산하고, 월별 총 주문량을 그래프로 출력한다. (x-axis: MM, y-axis: num_orders)

In [ ]:
query = ""
df = get_output(query)
df.plot(kind='bar', x='', y='')

**Question 3**: 총 결제 금액이 가장 많은 customer에 대한 정보 (customer_unique_id, total_payment, total_orders, customer_city, customer_state) 를 총 결제 금액에 대한 내림차순으로 정렬 후, 상위 5명의 정보를 출력한다. 결제 금액이 동일한 경우 customer_unique_id 기준 사전 순서로 구별한다.

In [ ]:
query = ""
print(get_output(query))

**Question 4-1**: 가장 많은 매출을 기록한 도시 (total_sales 가 가장 높은 도시) 를 매출의 내림차순으로 정렬하고, 상위 5개 도시에 대한 정보(city, state, total_orders, total_revenue)를 출력한다. 매출 측정시 구매자 (구매자 위치) 를 기준으로 매출을 측정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 4-2**: 가장 많은 매출을 기록한 주와 가장 적은 매출을 가지는 주의 이름 (max_state, min_state) 및 매출의 차를 순서대로 출력한다. 매출 측정시 구매자 (구매자 위치) 기준으로 매출을 측정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 5**: 각 state (customer 기준) 마다 total_num_orders, num_cities, num_delivered, num_not_delivered, delivery_success_rate, delivery_failure_rate 를 출력한다. 성공적으로 배송된 주문은 항상 "delivered" 상태로 가정한다.

In [ ]:
query = ""
print(get_output(query))

**Question 6**: 가격이 가장 높은 아이템 10개의 product_id, price, category 를 가격의 내림차순으로 출력한다.

In [ ]:
query = ""
print(get_output(query))

**Question 7**: 한 세션 (timestamp 기준) 에 3 종류 이상의 아이템을 구매한 customer 의 주소 (city, state) 를 출력한다.

In [ ]:
query = ""
print(get_output(query))

**Question 8**: 

각각의 주문(order)에는 여러 아이템이 포함된다. 평점이 4점 이상이며 "delivered" 된 주문들을 선별해, 주문에 포함된 아이템들의 평균 price가 가장 높은 / 낮은 주문의 order_id 를 찾는다. 이 때, order의 평균 price가 같을 경우 order_id의 오름차순으로 선정한다. 

각 주문을 찾은 후 평균 price가 가장 높은 주문에서는 주문 내에서 price가 가장 높은 아이템의 product_id 와 price 를, 가장 낮은 주문에서는 price가 가장 낮은 아이템의 product_id와 price를 찾는다. 

하나의 query를 통해 앞선 과정을 모두 수행하고, order_id, product_id, product_price column을 가진 table을 얻는다. 첫번째 행에는 금액이 높은 주문, 두번째 행에는 금액이 낮은 주문의 정보를 표시한다.

In [ ]:
query = ""
print(get_output(query))